In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
import os
import sys
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import numpy as np
from sklearn.metrics import f1_score
import random
import torch.nn.utils.prune
import re

In [ ]:
def process_tweet(tweet):

    tweet2 = re.sub(r'^RT[\s]+','', tweet)
   
    #remove hyperlinks
    tweet2 = re.sub(r'https?://[^\s\n\r]+', '', tweet2)
    
    #remove hashtag by removing the hast #sign from the word
    tweet2 = re.sub(r'#','',tweet2)
    
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    
    #convert into lower case
    tweets_clean = tweet.lower()

    return tweets_clean

In [ ]:
data = pd.read_csv("https://rbyakod-twitter-analysis.s3.amazonaws.com/data/processed/sentiment_label_data_sample.csv")

In [ ]:
#cleaning the tweets 
data["tweets"] = data["text"].map(lambda x : process_tweet(x))

In [ ]:
data["tweets"].sample(10)

In [ ]:
#splitting the dataset into training and test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.index.values, data.sentiment.values, test_size = 0.333, random_state=0)

In [ ]:
data["data_type"] = ['not_set']*data.shape[0]

In [ ]:
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_test, 'data_type'] = 'test'

In [ ]:
data.sample(10)

In [ ]:
# Encoding the data

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    return_tensors='pt'
                            
)

encoded_data_test = tokenizer.batch_encode_plus(
    data[data.data_type=='test'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    return_tensors='pt'
                            
)

In [ ]:
label_array = data["sentiment"].unique()
label_dict = dict(enumerate(label_array.flatten(), 1))
label_dict

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].sentiment.values)

# test dataset
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(data[data.data_type=='test'].sentiment.values)

In [ ]:
# Dividing datasets into train and test
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_test = TensorDataset(input_ids_test,attention_masks_test, labels_test)

In [ ]:
#Setting up BERT pretrained model
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False
    
)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4

dataloader_train = DataLoader(
   dataset_train, 
   sampler=RandomSampler(dataset_train),
   batch_size=batch_size)

dataloader_test = DataLoader(
   dataset_test, 
   sampler=RandomSampler(dataset_test),
   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
#setting hyper parameters
optimizer = torch.optim.AdamW(
 model.parameters(),
    lr=1e-5,
    eps=1e-8   
)

In [ ]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(
 optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    lables_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = pred_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Creating our Training Loop

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

In [ ]:
def evaluate(dataloader_test):
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_test:
        
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]
                 }
        with torch.no_grad():
            outputs = model(**inputs)
            
            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()
            
            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)
        
        loss_val_avg = loss_val_total/len(dataloader_test)
        
        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)
        
        return loss_val_avg, predictions, true_vals
                              

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train,
                        desc='epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)
    for batch in progress_bar:
        
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids' : batch[0],
            'attention_mask' : batch[1],
            'labels' : batch[2]
        }
        outputs = model(**inputs)
        
        optimizer.zero_grad()
        loss= outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        
        optimizer.step()
        
                                   
    print('training_loss : ',(loss.item()/len(batch)))    
    torch.save(model.state_dict(), f'BERT_ft_epoch{epoch}.model')    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')
  